## <center><strong><span style= 'color: #51fcc6'>Notebook </span>Describtion

Here we will try to extract more data from the describtions of each job using simple NER, POS & Regex expressions<br>
We will also need to clean the data after we extract it and make sure that there isn't any NANs

**What will I clean in the data:**
- Salaries data
- the Outliers
- Useless columns
- Tiny problems


**Things we will extract from the describtions:**
- Removing useless columns
- Cleaning linkedIn salary data
- Requeird years of experience
- Requeird programming languages

## <center><strong>Importing the <span style= 'color: #48e0dc'>Packeges</span>

In [14]:
import os
import re
import sys
import nltk
import random
import warnings
import matplotlib

import pandas             as pd
import numpy              as np
import seaborn            as sns
import matplotlib.pyplot  as plt

from io                   import StringIO 
from dotenv               import load_dotenv
from bs4                  import BeautifulSoup
from google.cloud         import bigquery
from wordcloud            import WordCloud
from IPython.display      import set_matplotlib_formats
from collections          import Counter

In [15]:
%matplotlib inline
load_dotenv()
plt.rcParams['font.family'] = 'Candara'

warnings.filterwarnings('ignore')

sys.path.insert(1, '../my_encrypter')
import encrypt

encrypt.decrypt_json_file('../credentials.json')

credentials_path :str = '../credentials.json'
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = credentials_path

FONT             :int  = 17
COLORS           :list = ['#51fcc6', '#48e0dc', '#5cd3f7', '#4895e0', '#517afc']
NUMERICS         :list = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64',
                          'uint16', 'uint32', 'uint64']

## <center><strong>Preparing the data <span style= 'color: #5cd3f7'>Extracting</span> 

In [16]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\FreeComp\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [17]:
project_id :str = 'data-jobs-analysis-db'
dataset_id :str = 'data_jobs_analysis_db'

client   = bigquery.Client(project= project_id)

linkedin_jobs    = client.query(f'SELECT * FROM {dataset_id}.linkedin_jobs;').to_dataframe()
upwork_profiles  = client.query(f'SELECT * FROM {dataset_id}.upwork_profiles;').to_dataframe()
guru_profiles    = client.query(f'SELECT * FROM {dataset_id}.guru_profiles;').to_dataframe()

In [18]:
linkedin_jobs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3455 entries, 0 to 3454
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   jobs_locations      3455 non-null   object
 1   listing_dates       3349 non-null   object
 2   jobs_titles         3455 non-null   object
 3   companies_names     3455 non-null   object
 4   jobs_links          3455 non-null   object
 5   describtion         3455 non-null   object
 6   location_type       3455 non-null   object
 7   employment_type     3455 non-null   object
 8   industry            3414 non-null   object
 9   reqierd_credential  3455 non-null   object
 10  country             3455 non-null   object
 11  job_title           3455 non-null   object
 12  total_jobs          3455 non-null   Int64 
dtypes: Int64(1), object(12)
memory usage: 354.4+ KB


<h4> <strong>Important note:</strong></h4>
We will need to use a string pattern search algorith so I will use <strong><code>Boyer Moore search</code></strong> to find the<br> strings more fast
and here's an image to explain it:<br>
<center><img src="https://www.researchgate.net/publication/337265181/figure/fig2/AS:825303362437121@1573779063161/Intuition-of-the-Boyer-Moore-search-procedure.png" alt="Intuition of the Boyer-Moore search procedure." itemprop="contentUrl" class="figure-details-image__main-image"></center>

In [19]:
def boyer_moore_search(pattern: str, text: str) -> dict:
    
    m              : int  = len(pattern)
    n              : int  = len(text)
    match          : list = []
    bad_char_table : dict = {}
    
    for i in range(m - 1):
        bad_char_table[pattern[i]] = m - i - 1
        
    i : int = m - 1
    
    while i < n:
        
        k : int = 0
        while k < m and pattern[m - 1 - k] == text[i - k]:
            k += 1
        if k == m:
            match.append(i - m + 1)
            i += m

            break
        else:
            char_shift = bad_char_table.get(text[i], m)
            i += max(1, char_shift)

    return match

def find_strings_in_string(string_list: list, target_string: str) -> list:

    matches = set()

    for s in string_list:
        if len(s) <= len(target_string):
            if boyer_moore_search(s, target_string):
                matches.add(s)

    return list(matches)

The function below is used to clean the `likedin_df` from the HTML tags so it's easier for the regex to read

In [20]:
def remove_html_tags(text):
    
    text_with_raw_html : str = BeautifulSoup(text, 'lxml').text
    text               : str = BeautifulSoup(text_with_raw_html, 'lxml').text

    return text

## <center><strong>Extracting features from <span style= 'color: #4895e0'>Jobs descriptions</span> 

In [21]:
def extract_salary(description) -> float:
    
    salary_regex = r'\$([0-9,.]+(?:K|k)?(?:\.\d+)?)(?:\s*(?:to|-)\s*\$([0-9,.]+(?:K|k)?(?:\.\d+)?))?'
    salary_matches = re.findall(salary_regex, description)

    
    if salary_matches:
        salary_range = tuple(float(value.replace(',', '').replace('.', '').lower().replace('k', '000')) for value in salary_matches[0] if value != '' )
        salary = sum(salary_range) / len(salary_range)
    else:
        salary = np.nan

    return salary

In [22]:
def extract_years(description) -> float:
    
    def convert_to_float(value):
        value = value.replace(',', '').replace('.', '').replace(' ', '')
        return float(value)
    
    years_regex   :str  = r'(\d+(?:[,.]\d+)*(?:\.\d+)?)\s*(?:\+|-)?\s*(?:years?|yrs?)'
    years_matches :list = re.findall(years_regex, description, re.IGNORECASE)

    years_list :list = [convert_to_float(value) for value in years_matches if convert_to_float(value) < 15]
    years      :int  =  np.mean(years_list)
    
    return years

In [23]:
def extract_programming_lang(description) -> list:

    languages  = ['Python', 'R lang', 'SQL', 'Kotlin', 'Java', 
                  'Java script', 'Type script', 'C#', 'C++',
                  'Rust', 'Js', 'Scala', 'HTML', 'CSS']


    languages  = [lang.lower() for lang in languages]
    languages  = find_strings_in_string(string_list=   languages,
                                        target_string= description.lower())

    return languages

In [24]:
def extract_skills(description) -> list:

    data_analysis_skills = [
        'Pandas', 'Excel',
        'NumPy', 'Matplotlib',
        'Seaborn', 'Tableau',
        'Power BI', 'Data querying',
        'Data mining', 'Data interpretation',
        'Data modeling', 'Data reporting',
        'Business intelligence', 'Data analytics',
        'Data validation', 'Data profiling',
        'Data aggregation', 'Data imputation',
        'Data manipulation', 'Data auditing',
        'Data quality management', 'SAS',
        'Data cleaning', 'Data visualization',
        'Statistical analysis', 'SPSS']

    data_science_skills = [
        'Machine learning', 'Deep learning', 'NLP',
        'Natural language processing', 'Computer vision',
        'Big data', 'Data wrangling',
        'Feature engineering', 'Predictive modeling',
        'Time series analysis', 'TensorFlow',
        'Keras', 'PyTorch',
        'Scikit-learn', 'Hadoop',
        'Spark', 'Data storytelling',
        'A/B testing', 'Data mining',
        'Data cleaning', 'Data visualization',
        'Data manipulation', 'Data pre-processing',
        'Data analysis', 'Data presentation',
        'Data-driven decision making', 'Model evaluation',
        'Model deployment']

    data_engineering_skills = [
        'Data warehousing', 'Data pipelines',
        'Database management', 'Data architecture',
        'Data integration', 'Apache Kafka',
        'Apache Airflow', 'Amazon Web Services',
        'Google Cloud Platform', 'Microsoft Azure',
        'Docker', 'Kubernetes',
        'Data security', 'Data governance',
        'Data scalability', 'Data storage',
        'Data migration', 'Data transformation',
        'Data orchestration', 'Data monitoring',
        'Data lake', 'AWS', 'GCP', 'ETL']

    data_entry_skills = [
        'Typing speed', 'Data accuracy',
        'Data entry software', 'Excel',
        'Google Sheets', 'Data verification',
        'Data organization', 'Attention to detail',
        'Time management', 'Data maintenance',
        'Database management', 'Copy typing',
        'Data review', 'Data formatting',
        'Data categorization', 'Data entry validation',
        'Data cleansing', 'Data input',
        'Data indexing', 'Data extraction',
        'Data capture', 'Data filing',
        'Data archiving']

    all_skills : list = (
        data_analysis_skills + data_science_skills +
        data_engineering_skills + data_entry_skills)
    
    all_skills = [skill.lower() for skill in all_skills]
    skills     = find_strings_in_string(string_list=   all_skills,
                                        target_string= description.lower())
    
    return skills

In [25]:
linkedin_jobs['describtion']      : pd.Series = linkedin_jobs['describtion'].apply(remove_html_tags)

linkedin_jobs['salary']           : pd.Series = linkedin_jobs['describtion'].apply(extract_salary)
linkedin_jobs['skills']           : pd.Series = linkedin_jobs['describtion'].apply(extract_skills)
linkedin_jobs['programming_lang'] : pd.Series = linkedin_jobs['describtion'].apply(extract_programming_lang)
linkedin_jobs['exp_years']        : pd.Series = linkedin_jobs['describtion'].apply(extract_years)

## <center><strong>Data <span style= 'color: #517afc'>Cleaning</span> 

In [26]:
def tucky_method(array: np.array, indecies= True) -> np.array:
    '''
    This function works with any list-like numerical object
    (don't work with pandas series's) and returns the indexes
    of the found outliers in the array.
    
    :Params: Takes only the series.
    :Returns: S list of the outliers indexes.
    '''
    
    Q3 = np.quantile(array, 0.75)
    Q1 = np.quantile(array, 0.25)
    IQR = Q3 - Q1
    
    upper_range = Q3 + (IQR * 1.5)
    lower_range = Q1 - (IQR * 1.5)
    
    outliers = [x for x in array if ((x < lower_range) | (x > upper_range))]
    print(f'Found {len(outliers)} outliers from {len(array)} length series!')
    
    return outliers

In [27]:
# Cleaning the salary data
linkedin_jobs['salary'] = linkedin_jobs['salary'].where((linkedin_jobs.salary < 70) |
                                                        (linkedin_jobs.salary > 10_000) &
                                                        (linkedin_jobs.salary > 5))

per_hour_or_year_map = lambda salary: 'per year' if salary > 10_000 else 'per hour' if salary < 100 else np.nan
linkedin_jobs['per_hour_or_year'] = linkedin_jobs['salary'].apply(per_hour_or_year_map)

In [29]:
# Removing useless columns
upwork_profiles.drop(['Unnamed__0', 'new_columns'], axis= 1, inplace= True)
guru_profiles.drop(['Unnamed__0', 'new_columns'], axis= 1, inplace= True)
linkedin_jobs.drop(['jobs_links'], axis= 1, inplace= True)

In [30]:
# Dropping outliers
outliers = tucky_method(linkedin_jobs[linkedin_jobs['per_hour_or_year'] == 'per year']['salary'].to_numpy())
outliers_indexes = np.array(*np.where(np.isin(linkedin_jobs['salary'], outliers)))
linkedin_jobs = linkedin_jobs.drop(outliers_indexes)

guru_profiles['country'] = guru_profiles['addresse'].str.split(',').str[-1]

Found 77 outliers from 964 length series!


## <center><strong>Saving the <span style= 'color: #4895e0'>Data</span> 

In [31]:
# Firstly i'll apply `extract_salary` function on guru minimum pay to get it as integer
guru_profiles['minimum_pay'] = guru_profiles['minimum_pay'].apply(extract_salary)

# And also i will exxtract some data from upwork
upwork_profiles['exp_years'] = upwork_profiles['describtion'].apply(extract_years)

TypeError: expected string or bytes-like object

In [32]:
dataframes = {
    'linkedin_jobs'   : linkedin_jobs,
    'guru_profiles'   : guru_profiles,
    'upwork_profiles' : upwork_profiles
}

for table_name, df in dataframes.items():

    table_id   = f'{project_id}.{dataset_id}.{table_name}'

    job_config = bigquery.LoadJobConfig(write_disposition= 'WRITE_TRUNCATE')
    job        = client.load_table_from_dataframe(df, table_id, job_config=job_config)

    job.result()
    print(f'DataFrame \'{table_name}\' uploaded as table \'{table_id}\' in BigQuery.')

encrypt.encrypt_json_file('../credentials.json')

DataFrame 'linkedin_jobs' uploaded as table 'data-jobs-analysis-db.data_jobs_analysis_db.linkedin_jobs' in BigQuery.
